# Seasonal Cycle Plotting

## Environment Settings

In [1]:
# filter some warning messages
import warnings
warnings.filterwarnings("ignore")

# supress some annoying warnings
import logging
urllib3_log = logging.getLogger("urllib3")
urllib3_log.setLevel(logging.CRITICAL)

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe

import copy

Dask Settings

In [32]:
from dask.distributed import Client
from dask_gateway import Gateway

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum = 1, maximum = 10)
 
client = Client(cluster, timeout="50s") 
cluster

In [ ]:
client.close()
cluster.close()

## Data Access - Read CMIP6 data from Google Cloud using intake
The Pangeo cmip6 catalog listing is [here](https://catalog.pangeo.io/browse/master/climate/cmip6_gcs/)

In [3]:
import intake
url = "https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml"
cat = intake.open_catalog(url)
col = cat.climate.cmip6_gcs()

# Actually this catalog links to the following csv file
# df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
# Another catagory from NCAR:
# url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
# col = intake.open_esm_datastore(url)

##################### old method ###################
## Using the csv file above
##
# def get_data(df, var, model, expe, freq):
#     try:
#         uri = df[(df.variable_id == var) & \
#                  (df.source_id == model) & \
#                  (df.experiment_id == expe) & \
#                  (df.table_id == freq)].zstore.values[0]
#         gcs = gcsfs.GCSFileSystem(token='anon')
#         ds = xr.open_zarr(gcs.get_mapper(uri), consolidated=True)
#         print(model, var, "found data")
#     except:
#         ds = []
#         #print(model, var, "no data")
#     return ds       

In [4]:
def rename_coords(ds):
    ds = ds.copy()
    """Rename all depth dim to `lev`"""
    if "olevel" in ds.coords:
        ds = ds.rename({"olevel": "lev"})
    if "lev_partial" in ds.coords:
        ds = ds.rename({"lev_partial": "lev"})
    """Rename all latitude, longitude dim to `lat`,`lon`"""
    if 'latitude' in ds.coords:
        ds = ds.rename({'longitude': 'lon', 'latitude': 'lat'})
    if 'nav_lat' in ds.coords:
        ds = ds.rename({'nav_lon': 'lon', 'nav_lat': 'lat'})
    if 'nav_lat' in ds.coords:
        ds = ds.rename({'nav_lon': 'lon', 'nav_lat': 'lat'})
    return ds

def search_data(col, models, var, expe, freq):
    datasets = {}
#     z_kwargs = {'consolidated': True, 'decode_times':False}
    for v in var:
        print(v)
        datasets_var = {}
        dcat = col.search(require_all_on = ["source_id"],
                          variable_id = v,
                          table_id=freq, 
                          experiment_id=expe,
                          source_id=models)
        print(dcat.unique(columns=["source_id"]))
        for model in models:
            for exp in expe:
                dataset_col = dcat.search(experiment_id = exp, source_id = model)             
                if not dataset_col.df.empty:
                    data_source =  dataset_col.search(member_id = 'r1i1p1f1')                    
                    if data_source.df.empty:
                        ds_member_id = dataset_col.df.iloc[0].member_id
                        data_source =  dataset_col.search(member_id = ds_member_id)
                    key0 = data_source.keys()[0]
                    datasets_var[key0] = data_source[key0]
        datasets[v] = datasets_var

    return datasets


#                     datasets[key0] = data_source(zarr_kwargs=z_kwargs, preprocess=rename_coords).to_dask()


#         z_kwargs = {'consolidated': True, 'decode_times':False}
#         dataset_dict = dcat.to_dataset_dict(zarr_kwargs=z_kwargs, preprocess=rename_coords)
#         for k in dataset_dict:
#             ds = dataset_dict[k].copy()
#             if 'member_id' in ds.dims: 
#                 try:
#                     ds = ds.sel(member_id='r1i1p1f1')
#                 except:
#                     ds = ds.isel(member_id = 0)
#             dataset_dict[k] = ds
#         dataset[v] = dataset_dict
#     return dataset 

In [5]:
# All models that have ocean ecology components
models = ['CESM2','CESM2-WACCM','CESM2-FV2','CESM2-WACCM-FV2',
          'GFDL-CM4','GFDL-ESM4','IPSL-CM6A-LR','MRI-ESM2-0',
          'GISS-E2-1-G','GISS-E2-1-G-CC','UKESM1-0-LL',
          'CanESM5','CanESM5-CanOE','NorESM2-LM','NorESM2-MM','NorCPM1',
          'MPI-ESM1-2-HR','MPI-ESM1-2-LR','MPI-ESM-1-2-HAM','MPI-ESM2-0',
          'MIROC-ES2l','CNRM-ESM2-1','ACCESS-ESM1-5']

# Ocean ecology variables 
variables = ['phyc','chl','phydiat','zooc','intpp','epc100',
             'fgco2','spco2','dfe','no3','si','rsntds']

experiments=["historical", "ssp585"] # return models with both experiments
frequencies=["Omon"]

In [28]:
### Test ###
models = ['CESM2', 'GFDL-ESM4']
variables = ['phyc', 'graz']
############
dataset_dict = search_data(col, models, variables, experiments, frequencies)

phyc
{'source_id': {'count': 1, 'values': ['GFDL-ESM4']}}
graz
{'source_id': {'count': 1, 'values': ['GFDL-ESM4']}}


In [14]:
dataset_dict['phyc']

{'CMIP.NCAR.CESM2-WACCM.historical.Omon.gn': <name: CMIP.NCAR.CESM2-WACCM.historical.Omon.gn, assets: 1,
 'ScenarioMIP.NCAR.CESM2-WACCM.ssp585.Omon.gn': <name: ScenarioMIP.NCAR.CESM2-WACCM.ssp585.Omon.gn, assets: 1,
 'CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gr': <name: CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gr, assets: 1,
 'ScenarioMIP.NOAA-GFDL.GFDL-CM4.ssp585.Omon.gr': <name: ScenarioMIP.NOAA-GFDL.GFDL-CM4.ssp585.Omon.gr, assets: 1,
 'CMIP.NOAA-GFDL.GFDL-ESM4.historical.Omon.gr': <name: CMIP.NOAA-GFDL.GFDL-ESM4.historical.Omon.gr, assets: 1,
 'ScenarioMIP.NOAA-GFDL.GFDL-ESM4.ssp585.Omon.gr': <name: ScenarioMIP.NOAA-GFDL.GFDL-ESM4.ssp585.Omon.gr, assets: 1}

## Data Pre-processing
    1. Regrid

In [29]:
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-90, 91, 1.0)),
                     'lon': (['lon'], np.arange(0, 361, 1.0))})

In [30]:
def func_select_time(ds):
    ds = ds.copy()
    ds = xr.decode_cf(ds)
    if ds.experiment_id == 'historical':
        start_year = '1991'
        end_year = '2010'
    elif ds.experiment_id == 'ssp585':
        start_year = '2081'
        end_year = '2100'
    ds = ds.sel(time=slice(start_year, end_year))
    return ds

def func_regrid(ds, ds_out):
    dsr =  xe.Regridder(ds, ds_out, 'bilinear', periodic=True, ignore_degenerate=True)
    dsr._grid_in = None
    dsr._grid_out = None
    dsr_out = dsr(ds)
    dsr.clean_weight_file()
    return dsr_out

In [ ]:
%%time

ds_regrid_dict = {}
z_kwargs = {'consolidated': True, 'decode_times':False}
for var in variables:
    print(var)
    dsr_dict_var = {}
    ds_dict = dataset_dict[var]
    if ds_dict:
        for k in ds_dict:
            ds_source = ds_dict[k]
            ds = ds_source(zarr_kwargs=z_kwargs, preprocess=rename_coords).to_dask()
            print(ds.source_id, ds.experiment_id)
            ds = func_select_time(ds)
            ds = ds[var]
            dsr_out0 = func_regrid(ds, ds_out)
            dsr_out = xr.Dataset({var:dsr_out0})
            
            dsr_dict_var[k] = dsr_out
    
    ds_regrid_dict[var] = dsr_dict_var

In [27]:
ds.time

<xarray.DataArray 'time' (time: 240)>
array([cftime.DatetimeNoLeap(2081-01-16 12:00:00),
       cftime.DatetimeNoLeap(2081-02-15 00:00:00),
       cftime.DatetimeNoLeap(2081-03-16 12:00:00), ...,
       cftime.DatetimeNoLeap(2100-10-16 12:00:00),
       cftime.DatetimeNoLeap(2100-11-16 00:00:00),
       cftime.DatetimeNoLeap(2100-12-16 12:00:00)], dtype=object)
Coordinates:
  * time     (time) object 2081-01-16 12:00:00 ... 2100-12-16 12:00:00
Attributes:
    axis:           T
    bounds:         time_bnds
    calendar_type:  noleap
    description:    Temporal mean
    long_name:      time
    standard_name:  time

In [17]:
xr.decode_cf(ds)

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 180, lev: 35, lon: 360, member_id: 1, time: 1032)
Coordinates:
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(360, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(180, 2), meta=np.ndarray>
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(35, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(1032, 2), meta=np.ndarray>
  * time       (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: bnds
Data variables:
    graz       (member_id, time, lev, lat, lon) float32 dask.array<chunksize=(1, 26, 35, 180, 360), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:             ScenarioMIP
    branch_method:           standard
    branch_time_in_child:    60225.0
    branch_time_in_parent:   60225.0
    comment:                 <null ref>
    contact:                 gfdl.climate.model.info@noaa.gov
    creation_date:           2019-06-19T04:04:19Z
    data_specs_version:      01.00.27
    experiment:              update of RCP8.5 based on SSP5
    experiment_id:           ssp585
    external_variables:      areacello volcello
    forcing_index:           1
    frequency:               mon
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.G...
    grid:                    ocean data regridded from tripolar - nominal 0.5...
    grid_label:              gr
    history:                 File was processed by fremetar (GFDL analog of C...
    initialization_index:    1
    institution:             National Oceanic and Atmospheric Administration,...
    institution_id:          NOAA-GFDL
    license:                 CMIP6 model data produced by NOAA-GFDL is licens...
    mip_era:                 CMIP6
    nominal_resolution:      1x1 degree
    parent_activity_id:      CMIP
    parent_experiment_id:    historical
    parent_mip_era:          CMIP6
    parent_source_id:        GFDL-ESM4
    parent_time_units:       days since 1850-1-1
    parent_variant_label:    r1i1p1f1
    physics_index:           1
    product:                 model-output
    realization_index:       1
    realm:                   ocnBgchem
    references:              see further_info_url attribute
    source:                  GFDL-ESM4 (2018):\natmos: GFDL-AM4.1 (Cubed-sphe...
    source_id:               GFDL-ESM4
    source_type:             AOGCM AER CHEM BGC
    status:                  2020-03-30;created; by gcs.cmip6.ldeo@gmail.com
    sub_experiment:          none
    sub_experiment_id:       none
    table_id:                Omon
    title:                   NOAA GFDL GFDL-ESM4 model output prepared for CM...
    tracking_id:             hdl:21.14100/2aba7487-06cc-4d0c-b5e2-f28e028dcc3...
    variable_id:             graz
    variant_info:            N/A
    variant_label:           r1i1p1f1
    intake_esm_varname:      graz
    intake_esm_dataset_key:  ScenarioMIP.NOAA-GFDL.GFDL-ESM4.ssp585.Omon.gr

    2. Detrend

In [ ]:
def select_region(ds, region_no):
    ### region_no:  region number. 0 - Whole world, 1 - PAPA, 2 - NAPA
    ds = ds.copy()
    if region_no == 1:
        ds = ds.sel(lat=slice(45,50)).sel(lon=slice(220,230))
    if region_no == 2:
        ds = ds.sel(lat=slice(45,50)).sel(lon=slice(325,335))
    return ds   

In [ ]:
from scipy.signal import detrend

def func_detrend(ds, var):
    dsm = ds[var].mean(dim='time', skipna=1)
    ds_anom = ds[var] - dsm
    ds_anom.load()
    ds_anom_detrended = xr.apply_ufunc(detrend, ds_anom, kwargs={'axis': 0}).where(~ds_anom.isnull())
    ds_detrend = ds_anom_detrended + dsm
    
    return ds_detrend

In [ ]:
%%time

ds_dr_mean = {}
for var in variables:
    dsr_dict = ds_regrid_dict[var]
    ds_var = {}
    if dsr_dict:
        for k in dsr_dict:
            ds = dsr_dict[k]
            ds = select_region(ds, 1)
            ds = func_detrend(ds, var)
            ds = ds_detrend.groupby('time.month').mean('time', skipna=1).mean('lon', skipna=1).mean('lat', skipna=1).values
            ds_var[k] = ds
        
    ds_dr_mean[var] = ds_var


## Plotting

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
xr.set_options(display_style="html")  #display dataset nicely 
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina'  #enabling 2x PNG matplotlib plots in the IPython notebook

In [ ]:
# temp!
Time_months = ('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec');
Colors = [[1, 1, 0], [1, 0.8, 0], [1, 0.6, 0], 
          [1, 0.4, 0], [1, 0, 0], [1, 0.8, 1],
          [1, 0.5, 1], [1, 0, 1], [0, 0, 0.5],
          [0, 0, 1], [0, 0.5, 1], [0, 0.5, 0], 
          [0, 1, 0], [0, 0.6, 0], [0.5, 0.2, 0]]   
# color order  = ['CESM2', 'GFDL-CM4', 'GFDL-ESM4', 'CESM2-WACCM', 'IPSL-CM6A-LR', 'MRI-ESM2-0',
#           'GISS-E2-1-G', 'GISS-E2-1-G-CC', 'UKESM1-0-LL',  'CanESM5', 'NorESM2-LM', 
#           'MPI-ESM1-2-HR', 'NorCPM1', 'MIROC-ES2L', 'CNRM-ESM2-1']

In [ ]:
plot_titles=['Biomass (mol/m3)','Chlorophyll (kg/m3)','Diatom (mol/m3)', 
             'Zooplankton (mol/m3)','Primary Production (mol/m2/s)','Export Production at 100m (mol/m2/s)',
             'Air-sea CO2 flux (donward +) (gram C /m2/year)','SpCO2 (ppm)', 'Iron - Fe (mol/m3)', 
             'Nitrate - NO3 (mol/m3)','Silica - Si (mol/m3)', 'Light - Surface Downward SW Radiation (W/m2)']

In [ ]:
%%time

plt.figure(figsize=(8.5, 5))
l = []
for model in models:  
    if isinstance(papa_dic[model], list):
        continue
    else:
        s = papa_dic[model]
        if model == 'CNRM-ESM2-1' or model == 'IPSL-CM6A-LR':
            if var == 'chl':
                s = papa_dic[model] * 0.001        # need more infromation
        if model == 'GISS-E2-1-G' or model == 'GISS-E2-1-G-CC':
            if var == 'intpp':
                s = -papa_dic[model]               # need more infromation
            if var == 'fgco2':
                s = papa_dic[model]*(10**(-10))    # need more infromation
        
        plt.plot(np.arange(1, 13), s, label=str(model), marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
        l.append(str(model))
        
plt.subplots_adjust(left=0.12, bottom=0.05, right=0.80, top=0.92, hspace=0.3, wspace=0.02)
plt.xticks(np.arange(1, 13), Time_months)
plt.legend(l,loc='center left', bbox_to_anchor=(1, 0.5))
savename = var + '_papa.png'
plt.savefig(savename, dpi=600)    

**Notice** : 'CNRM-ESM2-1', 'IPSL-CM6A-LR':'chl'; 'GISS-E2-1-G', 'GISS-E2-1-G-CC': 'intpp', 'fgco2', 'dfe'

In [ ]:
%%time

fig = plt.figure(figsize=(18, 11))
ncol = 3
nrow = 4
i = 1
for var in var_names:
    
    ax = plt.subplot(ncol,nrow,i)
    i = i + 1
    
    mi = 0
    for model in data_np:
        dm = data_np[model]
        if var in dm:
            s = dm[var]
            if model == 'CNRM-ESM2-1' or model == 'IPSL-CM6A-LR':
                if var == 'chl':
                    s = dm[var] * 0.001        # need more infromation
            if model == 'GISS-E2-1-G' or model == 'GISS-E2-1-G-CC':
                if var == 'intpp':
                    s = -dm[var]               # need more infromation
                if var == 'fgco2':
                    s = dm[var]*(10**(-10))    # need more infromation
                if var == 'dfe':
                    s = dm[var] * 0.001        # need more infromation
                    
            plt.plot(np.arange(1, 13), s, c=Colors[mi], label=str(model), marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
        else:
            plt.plot([], [], c=Colors[mi], label=str(model), marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
            
        mi = mi +1
    #plt.subplots_adjust(left=0.12, bottom=0.05, right=0.80, top=0.92, hspace=0.3, wspace=0.02)
    plt.xticks(np.arange(1, 13), Time_months)
    ax.ticklabel_format(axis='y', style='sci', scilimits=(-3,1000))
    plt.title(plot_titles[i-2], fontsize=12, y=1.05)
    
    
plt.subplots_adjust(left=0.035, bottom=0.04, right=0.90, top=0.91, hspace=0.25, wspace=0.20) # the amount of height/width reserved for space between subplots

fig.legend(models, loc='lower right', bbox_to_anchor=(0.997, 0.33)) 
plt.suptitle('CMIP6 Model average seasonality for North Pacific [45N-50N, 140W-150W] - '+ str(start_year) + '-' + str(end_year), fontsize=18)

plt.savefig('MultiVar_NP_new.png', dpi=600)    


In [ ]:
%%time

fig = plt.figure(figsize=(18, 11))
ncol = 3
nrow = 4
i = 1

for var in var_names:
    
    ax = plt.subplot(ncol,nrow,i)
    i = i + 1
    
    mi = 0
    for model in data_na:
        xmon = np.arange(1, 13)
        dm = data_na[model]
        if var in dm:                
            s = dm[var]
            if model == 'CNRM-ESM2-1' or model == 'IPSL-CM6A-LR':
                if var == 'chl':
                    s = dm[var] * 0.001        # need more infromation
            if model == 'GISS-E2-1-G' or model == 'GISS-E2-1-G-CC':
                #if var == 'phyc' or var == 'chl' or var == 'phydiat' or var == 'zooc' or var == 'no3' or var == 'si' or var == 'dfe':
                    #s = []
                    #xmon = []
                    
                if var == 'intpp':
                    s = -dm[var]               # need more infromation
                if var == 'fgco2':
                    s = dm[var]*(10**(-10))    # need more infromation
                if var == 'dfe':
                    #s = dm[var] * 0.001        # need more infromation
                    s = []
                    xmon = []
                    
            plt.plot(xmon, s, c=Colors[mi], label=str(model), marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
        else:
            plt.plot([], [], c=Colors[mi], label=str(model), marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
            
        mi = mi +1
    #plt.subplots_adjust(left=0.12, bottom=0.05, right=0.80, top=0.92, hspace=0.3, wspace=0.02)
    plt.xticks(np.arange(1, 13), Time_months)
    ax.ticklabel_format(axis='y', style='sci', scilimits=(-3,1000))
    plt.title(plot_titles[i-2], fontsize=12, y=1.05)
    
    
plt.subplots_adjust(left=0.035, bottom=0.04, right=0.90, top=0.91, hspace=0.25, wspace=0.20) # the amount of height/width reserved for space between subplots

fig.legend(models, loc='lower right', bbox_to_anchor=(0.997, 0.35)) 
plt.suptitle('CMIP6 Model average seasonality for North Atlantic [45N-50N, 25W-35W] - '+ str(start_year) + '-' + str(end_year), fontsize=18)

plt.savefig('MultiVar_NA.png', dpi=600)    


In [ ]:
plot_ranges=[[-0.001,0.02], [-0.1e-6,5e-6], [-0.0005, 0.0175], [-0.0005,0.013], 
             [-0.05e-6,1.4e-6], [-0.1e-7,3.5e-7], [-30,85], [270,460],  
             [-0.1e-7,1e-6], [-0.001,0.012], [-0.001,0.016], [0,250]]

In [ ]:
%%time

fig = plt.figure(figsize=(18, 11))
ncol = 3
nrow = 4
i = 0
for var in var_names:
    
    
    i = i + 1
    ax = plt.subplot(ncol,nrow,i)
    
    mi = 0
    for model in models:
        
        ofn = 'save/' + str(model) + '_' + str(var) + '_np.txt'
        if os.path.exists(ofn):
            
            dm = []
            with open(ofn, "r") as rf:
                for line in rf:
                    dm.append(float(line.strip()))
            
            xmon = np.arange(1, 13)
            s = np.array(dm)
            if model == 'CNRM-ESM2-1' or model == 'IPSL-CM6A-LR':
                if var == 'chl':
                    s = s * 0.001        # need more infromation
            if model == 'GISS-E2-1-G' or model == 'GISS-E2-1-G-CC':
                if var == 'intpp':
                    s = -s             # need more infromation
                if var == 'fgco2':
                    s = s*(10**(-10))    # need more infromation
                if var == 'dfe':
                    xmon =[]
                    s = []        # need more infromation
                    
            plt.plot(xmon, s, c=Colors[mi], label=str(model), 
                     marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
        else:
            plt.plot([], [], c=Colors[mi], label=str(model), 
                     marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
            
        mi = mi +1
    #plt.subplots_adjust(left=0.12, bottom=0.05, right=0.80, top=0.92, hspace=0.3, wspace=0.02)
    plt.ylim(plot_ranges[i-1])
    plt.xticks(np.arange(1, 13), Time_months)
    ax.ticklabel_format(axis='y', style='sci', scilimits=(-3,1000))
    plt.title(plot_titles[i-1], fontsize=12, y=1.05)
    
    
plt.subplots_adjust(left=0.035, bottom=0.04, right=0.90, top=0.91, hspace=0.25, wspace=0.20) # the amount of height/width reserved for space between subplots

fig.legend(models, loc='lower right', bbox_to_anchor=(0.997, 0.33)) 
plt.suptitle('CMIP6 Model average seasonality for North Pacific [45N-50N, 140W-150W] - '+ str(start_year) + '-' + str(end_year), fontsize=18)

plt.savefig('MultiVar_NP_new.png', dpi=600)    


In [ ]:
%%time

fig = plt.figure(figsize=(18, 11))
ncol = 3
nrow = 4
i = 0
for var in var_names:
    
    i = i + 1
    ax = plt.subplot(ncol,nrow,i)
    
    mi = 0
    for model in models:
        
        ofn = 'save/' + str(model) + '_' + str(var) + '_na.txt'
        if os.path.exists(ofn):
            
            dm = []
            with open(ofn, "r") as rf:
                for line in rf:
                    dm.append(float(line.strip()))
            
            s = np.array(dm)
            xmon = np.arange(1, 13)
            
            if model == 'CNRM-ESM2-1' or model == 'IPSL-CM6A-LR':
                if var == 'chl':
                    s = np.array(dm) * 0.001        # need more infromation
            if model == 'GISS-E2-1-G' or model == 'GISS-E2-1-G-CC':
                if var == 'intpp':
                    s = -np.array(dm)            # need more infromation
                if var == 'fgco2':
                    s = np.array(dm)*(10**(-10))    # need more infromation               
                if var == 'dfe':                   
                    #s = np.array(dm) * 0.001        # need more infromation
                    s = []
                    xmon = []
                    
            plt.plot(xmon, s, c=Colors[mi], label=str(model), 
                     marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
        else:
            plt.plot([], [], c=Colors[mi], label=str(model), 
                     marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
            
        mi = mi +1
    #plt.subplots_adjust(left=0.12, bottom=0.05, right=0.80, top=0.92, hspace=0.3, wspace=0.02)
    plt.ylim(plot_ranges[i-1])
    plt.xticks(np.arange(1, 13), Time_months)
    ax.ticklabel_format(axis='y', style='sci', scilimits=(-3,1000))
    plt.title(plot_titles[i-1], fontsize=12, y=1.05)
    
    
plt.subplots_adjust(left=0.035, bottom=0.04, right=0.90, top=0.91, hspace=0.25, wspace=0.20) # the amount of height/width reserved for space between subplots

fig.legend(models, loc='lower right', bbox_to_anchor=(0.997, 0.33)) 
plt.suptitle('CMIP6 Model average seasonality for North Atlantic [45N-50N, 25W-35W] - ' + str(start_year) + '-' + str(end_year), fontsize=18)

plt.savefig('MultiVar_NA_new.png', dpi=600)    


# SpCO2 separation

In [ ]:
models = ['NorCP1']

In [ ]:
%%time

for model in models:
    sparay = get_data(df, 'spco2', model, expe, freq)
    #print(sparay)
    if isinstance(sparay, xr.Dataset):
        staray = get_data(df, 'tos', model, expe, freq)
        #print(staray)
        if not isinstance(staray, xr.Dataset):
            print(model, 'no tos, try thetao')
            staray = get_data(df, 'thetao', model, expe, freq)
            #print(staray)
            if not isinstance(staray, xr.Dataset):
                print(model, 'no sst data')
                continue
    else:
        continue
    
    dsp = sparay.sel(time=slice(start_year, end_year))['spco2']
    
    if 'thetao' in staray:
        if 'lev_partial' in staray.coords:
            staray = staray.isel(lev_partial=0)
        if 'olevel' in staray.coords:
            staray = staray.isel(olevel=0)
        if 'lev' in staray.coords:
            staray = staray.isel(lev=0)
        if 'rho' in staray.coords:
            staray = staray.isel(rho=0)
        dst = staray.sel(time=slice(start_year, end_year))['thetao']
        
        #dst = dst.rename({'thetao':'tos'})
    else:
        dst = staray.sel(time=slice(start_year, end_year))['tos']
        
    if dst.shape[0] != 240:
        continue
    
    
    if 'latitude' in dsp.coords:
        dsp = dsp.rename({'longitude': 'lon', 'latitude': 'lat'})
        dst = dst.rename({'longitude': 'lon', 'latitude': 'lat'})
    if 'nav_lat' in dsp.coords:
        dsp = dsp.rename({'nav_lon': 'lon', 'nav_lat': 'lat'})
        dst = dst.rename({'nav_lon': 'lon', 'nav_lat': 'lat'})
    
    mrp = xe.Regridder(dsp, ds_out, 'bilinear', periodic=True, ignore_degenerate=True)
    mrp._grid_in = None
    mrp._grid_out = None
    dsp_out = mrp(dsp)
    mrt = xe.Regridder(dst, ds_out, 'bilinear', periodic=True, ignore_degenerate=True)
    mrt._grid_in = None
    mrt._grid_out = None
    dst_out = mrt(dst)
    
    m_rg = xr.Dataset({'spco2':dsp_out, 'sst':dst_out})
    

    if m_rg['spco2'].shape[0]==240:
        if m_rg['sst'].shape[0]==240:
            print(m_rg['sst'].shape)
            
            ## PAPA site ##
            d_np = m_rg.sel(lat=slice(45,50)).sel(lon=slice(210,220))    
            
            msp_np = d_np['spco2'].mean(dim='time',skipna=1) 
            anomsp_np = d_np['spco2'] - msp_np
            mst_np = d_np['sst'].mean(dim='time',skipna=1)
            anomst_np = d_np['sst'] - mst_np
            
            anomsp_np.load()           
            anomsp_detrended_np = xr.apply_ufunc(detrend, anomsp_np.fillna(0), 
                                                 kwargs={'axis': 0}).where(~anomsp_np.isnull())
            dtsp_np = anomsp_detrended_np + msp_np   
            dtsp_np = dtsp_np / 0.101325 
            
            anomst_np.load()
            anomst_detrended_np = xr.apply_ufunc(detrend, anomst_np.fillna(0), 
                                                 kwargs={'axis': 0}).where(~anomst_np.isnull())
            dtst_np = anomst_detrended_np + mst_np      
            
            sp_temp_np = np.mean(dtsp_np) * np.exp(0.0423 * anomst_detrended_np)
            sp_ntemp_np = dtsp_np * np.exp( 0.0423 * (- anomst_detrended_np))       
            
            monsp_t_np = sp_temp_np.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values
            monsp_nt_np = sp_ntemp_np.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values
            sst_np = dtst_np.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values               
            
            savet_np = 'save/' + model + '_pco2-t_np.txt'
            with open(savet_np, 'w') as nptf:
                for idata in monsp_t_np:
                    nptf.write(str(idata) +"\n")        
            
            savent_np = 'save/' + model + '_pco2-nt_np.txt'
            with open(savent_np, 'w') as npntf:
                for idata in monsp_nt_np:
                    npntf.write(str(idata) +"\n")
            
            savesst_np = 'save/' + model + '_sst_np.txt'
            with open(savesst_np, 'w') as npsstf:
                for idata in sst_np:
                    npsstf.write(str(idata) +"\n")
                    
            
            ## NABA site ##
            d_na = m_rg.sel(lat=slice(45,50)).sel(lon=slice(325,335))
            
            msp_na = d_na['spco2'].mean(dim='time',skipna=1)
            anomsp_na = d_na['spco2'] - msp_na
            mst_na = d_na['sst'].mean(dim='time',skipna=1)
            anomst_na = d_na['sst'] - mst_na
            
            anomsp_na.load()
            anomsp_detrended_na = xr.apply_ufunc(detrend, anomsp_na.fillna(0), kwargs={'axis': 0}).where(~anomsp_na.isnull())
            dtsp_na = anomsp_detrended_na + msp_na
            dtsp_na = dtsp_na / 0.101325 
            
            anomst_na.load()
            anomst_detrended_na = xr.apply_ufunc(detrend, anomst_na.fillna(0), kwargs={'axis': 0}).where(~anomst_na.isnull())
            dtst_na = anomst_detrended_na + mst_na
            
            sp_temp_na = np.mean(dtsp_na) * np.exp(0.0423 * anomst_na)
            sp_ntemp_na = dtsp_na * np.exp( 0.0423 * (- anomst_na))
            
            monsp_t_na = sp_temp_na.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values
            monsp_nt_na = sp_ntemp_na.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values
            sst_na = dtst_na.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values
            
            savet_na = 'save/' + model + '_pco2-t_na.txt'
            with open(savet_na, 'w') as natf:
                for idata in monsp_t_na:
                    natf.write(str(idata) +"\n")
            
            savent_na = 'save/' + model + '_pco2-nt_na.txt'
            with open(savent_na, 'w') as nantf:
                for idata in monsp_nt_na:
                    nantf.write(str(idata) +"\n")
            
            savesst_na = 'save/' + model + '_sst_na.txt'
            with open(savesst_na, 'w') as nasstf:
                for idata in sst_na:
                    nasstf.write(str(idata) +"\n")
    
    
    mrp.clean_weight_file()
    mrt.clean_weight_file()

In [ ]:
pvars=['spco2', 'pco2-t', 'pco2-nt',
       'chl', 'sst', 'phyc',
       'dfe','no3']
ptitles=['pCO2 (ppm)', 'pCO2-T (ppm)', 'pCO2-nonT (ppm)', 
         'Chlorophyll (kg/m3)', 'SST (C)', 'Biomass (mol/m3)',
         'Iron - Fe (mol/m3)', 'Nitrate - NO3 (mol/m3)']  
pranges=[[270, 485], [270, 485], [270, 485], 
         [-0.1e-6, 5e-6], [4, 20], [-0.001,0.02],
         [-0.1e-6, 1e-6], [-0.001, 0.012]]

In [ ]:
%%time

fig = plt.figure(figsize=(15, 11))
ncol = 3
nrow = 3
i = 0
for var in pvars:
    
    i = i + 1
    ax = plt.subplot(ncol,nrow,i)
    
    mi = 0
    for model in models:
        
        ofn = 'save/' + str(model) + '_' + str(var) + '_np.txt'
        if os.path.exists(ofn):
            
            dm = []
            with open(ofn, "r") as rf:
                for line in rf:
                    dm.append(float(line.strip()))
            
            xmon = np.arange(1, 13)
            s = np.array(dm)
            if model == 'CNRM-ESM2-1' or model == 'IPSL-CM6A-LR':
                if var == 'chl':
                    s = s * 0.001        # need more infromation
            if model == 'GISS-E2-1-G' or model == 'GISS-E2-1-G-CC':
                if var == 'intpp':
                    s = -s             # need more infromation
                if var == 'fgco2':
                    s = s*(10**(-10))    # need more infromation
                if var == 'dfe':
                    xmon =[]
                    s = []        # need more infromation
                    
            plt.plot(xmon, s, c=Colors[mi], label=str(model), 
                     marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
        else:
            plt.plot([], [], c=Colors[mi], label=str(model), 
                     marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
            
        mi = mi +1
    #plt.subplots_adjust(left=0.12, bottom=0.05, right=0.80, top=0.92, hspace=0.3, wspace=0.02)
    plt.ylim(pranges[i-1])
    plt.xticks(np.arange(1, 13), Time_months)
    ax.ticklabel_format(axis='y', style='sci', scilimits=(-3,1000))
    plt.title(ptitles[i-1], fontsize=12, y=1.05)
    
    
plt.subplots_adjust(left=0.035, bottom=0.04, right=0.90, top=0.91, hspace=0.25, wspace=0.20) # the amount of height/width reserved for space between subplots

fig.legend(models, loc='lower right', bbox_to_anchor=(0.8, 0.01)) 
plt.suptitle('CMIP6 SpCO2 separation and seasonal cylces - North Pacific [45N-50N, 140W-150W] - '+ str(start_year) + '-' + str(end_year), fontsize=18)

plt.savefig('pCO2_sep_NP.png', dpi=600)    


In [ ]:
%%time

fig = plt.figure(figsize=(15, 11))
ncol = 3
nrow = 3
i = 0
for var in pvars:
    
    i = i + 1
    ax = plt.subplot(ncol,nrow,i)
    
    mi = 0
    for model in models:
        
        ofn = 'save/' + str(model) + '_' + str(var) + '_na.txt'
        if os.path.exists(ofn):
            
            dm = []
            with open(ofn, "r") as rf:
                for line in rf:
                    dm.append(float(line.strip()))
            
            s = np.array(dm)
            xmon = np.arange(1, 13)
            
            if model == 'CNRM-ESM2-1' or model == 'IPSL-CM6A-LR':
                if var == 'chl':
                    s = np.array(dm) * 0.001        # need more infromation
            if model == 'GISS-E2-1-G' or model == 'GISS-E2-1-G-CC':
                if var == 'intpp':
                    s = -np.array(dm)            # need more infromation
                if var == 'fgco2':
                    s = np.array(dm)*(10**(-10))    # need more infromation               
                if var == 'dfe':                   
                    #s = np.array(dm) * 0.001        # need more infromation
                    s = []
                    xmon = []
                    
            plt.plot(xmon, s, c=Colors[mi], label=str(model), 
                     marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
        else:
            plt.plot([], [], c=Colors[mi], label=str(model), 
                     marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
            
        mi = mi +1
    #plt.subplots_adjust(left=0.12, bottom=0.05, right=0.80, top=0.92, hspace=0.3, wspace=0.02)
    plt.ylim(pranges[i-1])
    plt.xticks(np.arange(1, 13), Time_months)
    ax.ticklabel_format(axis='y', style='sci', scilimits=(-3,1000))
    plt.title(ptitles[i-1], fontsize=12, y=1.05)
    
    
plt.subplots_adjust(left=0.035, bottom=0.04, right=0.90, top=0.91, hspace=0.25, wspace=0.20) # the amount of height/width reserved for space between subplots

fig.legend(models, loc='lower right', bbox_to_anchor=(0.8, 0.01)) 
plt.suptitle('CMIP6 SpCO2 separation and seasonal cylces - North Atlantic [45N-50N, 25W-35W] - ' + str(start_year) + '-' + str(end_year), fontsize=18)

plt.savefig('pCO2_sep_NA.png', dpi=600)    


# Average parameter value

In [ ]:
var_names=['phyc','chl','phydiat','zooc',
           'intpp','epc100','sst','mld',
           'fgco2','spco2','pco2-t','pco2-nt',
           'dfe','no3','si','rsntds']
plot_titles=['Biomass (mol/m3)','Chlorophyll (kg.m-3)','Diatom (mol/m3)', 'Zooplankton (mol/m3)',
             'Primary Prod. (mol.m-2.s-1)','Export Prod. (mol.m-2.s-1)',   'SST (C)',   'MLD (m)', 
             'Air-sea CO2 flux(gram C /m2/year)', 'SpCO2 (ppm)', 'SpCO2-T (ppm)', 'SpCO2-nonT (ppm)', 
             'Iron - Fe (mol/m3)', 'Nitrate - NO3 (mol/m3)','Silica - Si (mol/m3)', 'Light (W/m2)']  
plot_ranges=[[-0.0005,0.005], [-0.1e-6,1e-6],  [-0.0005, 0.0045],  [-0.001,0.005], 
             [-0.05e-6,0.6e-6], [-0.1e-7,1.5e-7], [6,15], [-1,450], 
             [-5,60], [310,380], [310,380], [310,380], 
             [-0.1e-6,1e-6], [-0.001,0.01], [-0.0025,0.015], [90,160]]

In [ ]:
%%time

fig = plt.figure(figsize=(15, 12.5))
ncol = 4
nrow = 4
i = 0
for var in var_names:
    i = i + 1
    if var == 'mld':
        continue
    
    ax = plt.subplot(ncol,nrow,i)
    
    mi = 0
    hs = []
    for model in models:
        
        ofnp = 'save/' + str(model) + '_' + str(var) + '_np.txt'
        ofna = 'save/' + str(model) + '_' + str(var) + '_na.txt'
        if os.path.exists(ofnp):
            if os.path.exists(ofna):
                
                np_data = []
                with open(ofnp, "r") as rfnp:
                    for line in rfnp:
                        np_data.append(float(line.strip()))
                        
                na_data = []
                with open(ofna, "r") as rfna:
                    for line in rfna:
                        na_data.append(float(line.strip()))
                        
                np_value = np.array(np_data)
                na_value = np.array(na_data)
                if model == 'CNRM-ESM2-1' or model == 'IPSL-CM6A-LR':
                    if var == 'chl':
                        np_value = np_value * 0.001        # need more infromation
                        na_value = na_value * 0.001
                if model == 'GISS-E2-1-G' or model == 'GISS-E2-1-G-CC':
                    if var == 'intpp':
                        np_value = -np_value             # need more infromation
                        na_value = -na_value
                    if var == 'fgco2':                  
                        np_value = np_value*(10**(-10))    # need more infromation
                        na_value = na_value*(10**(-10))
                    if var == 'dfe':
                        np_value = []
                        na_value = []        # need more infromation                                           
                        
                h = plt.scatter(np.mean(np_value), np.mean(na_value), s=100, marker='o', edgecolor = 'black',
                            color=Colors[mi], label=str(model))
        else:
            h = plt.scatter([], [], s=100, marker='o', edgecolor = 'black',
                        color=Colors[mi], label=str(model))
            
        mi = mi +1
        hs.append(h)
    #plt.subplots_adjust(left=0.12, bottom=0.05, right=0.80, top=0.92, hspace=0.3, wspace=0.02)
    
    plt.ylim(plot_ranges[i-1])
    plt.xlim(plot_ranges[i-1])
    #plt.xticks(np.arange(1, 13), Time_months)
    ax.ticklabel_format(axis='both', style='sci', scilimits=(-2,1000))
    plt.title(plot_titles[i-1], fontsize=12.5, y=1.05)
    plt.plot([-1000,1000],[-1000,1000], 'k--', linewidth=0.75)
    
    
plt.subplots_adjust(left=0.035, bottom=0.04, right=0.88, top=0.91, hspace=0.35, wspace=0.25) # the amount of height/width reserved for space between subplots

fig.legend(hs, models, loc='lower right', bbox_to_anchor=(0.999, 0.03)) 
plt.suptitle('CMIP6 models - Average value for North Pacific (x-axis) and North Atlantic (y-axis) - '+ str(start_year) + '-' + str(end_year), fontsize=18)

plt.savefig('NA_NP_average.png', dpi=600)    


In [ ]:
mmranges = [[-1,7.5], [-1,7.5],  [-1,7.5],  [-1,5], 
             [-1,5], [-1,7.5], [0,2], [-1,450], 
             [-1,10], [-0.1,0.8], [-0.1,0.8], [-0.1,0.8], 
             [-0.1,2.5], [-0.1,4], [-0.1,2.5], [-0.1,2.2]]

In [ ]:
# Behzad's range
mmranges = [[-0.01,9], [-0.01,9], [-0.01,9], [-0.01,9],
            [-0.01,4.7], [-0.01,4.7], [-0.01,1.3], [-1,9],     
            [-1,25], [-0.01,0.5], [-0.01,0.5], [-0.01,0.5], 
            [-0.01,2.3], [-0.01,4], [-0.01,2.3], [-0.01,2.3]]

In [ ]:
%%time

fig = plt.figure(figsize=(15, 12.5))
ncol = 4
nrow = 4
i = 0
for var in var_names:
    i = i + 1
    if var == 'mld':
        continue
    
    ax = plt.subplot(ncol,nrow,i)
    
    mi = 0
    hs = []
    for model in models:
        
        ofnp = 'save/' + str(model) + '_' + str(var) + '_np.txt'
        ofna = 'save/' + str(model) + '_' + str(var) + '_na.txt'
        if os.path.exists(ofnp):
            if os.path.exists(ofna):
                
                np_data = []
                with open(ofnp, "r") as rfnp:
                    for line in rfnp:
                        np_data.append(float(line.strip()))
                        
                na_data = []
                with open(ofna, "r") as rfna:
                    for line in rfna:
                        na_data.append(float(line.strip()))
                        
                np_value = np.array(np_data)
                na_value = np.array(na_data)
                if model == 'CNRM-ESM2-1' or model == 'IPSL-CM6A-LR':
                    if var == 'chl':
                        np_value = np_value * 0.001        # need more infromation
                        na_value = na_value * 0.001
                if model == 'GISS-E2-1-G' or model == 'GISS-E2-1-G-CC':
                    if var == 'intpp':
                        np_value = -np_value             # need more infromation
                        na_value = -na_value
                    if var == 'fgco2':                  
                        np_value = np_value*(10**(-10))    # need more infromation
                        na_value = na_value*(10**(-10))
                    if var == 'dfe':
                        np_value = []
                        na_value = []        # need more infromation                                           
                
                if np.shape(np_value)[0] > 0:
                    np_plot = (np.nanmax(np_value) - np.nanmin(np_value)) / np.nanmean(np_value)
                    na_plot = (np.nanmax(na_value) - np.nanmin(na_value)) / np.nanmean(na_value)
                    
                    h = plt.scatter(np_plot, na_plot, s=100, marker='o', edgecolor = 'black',
                            color=Colors[mi], label=str(model))
                else:
                    h = plt.scatter([], [], s=100, marker='o', edgecolor = 'black',
                            color=Colors[mi], label=str(model))
        else:
            h = plt.scatter([], [], s=100, marker='o', edgecolor = 'black',
                        color=Colors[mi], label=str(model))
            
        mi = mi +1
        hs.append(h)
    #plt.subplots_adjust(left=0.12, bottom=0.05, right=0.80, top=0.92, hspace=0.3, wspace=0.02)
    plt.plot([-1000,1000],[-1000,1000], 'k--', linewidth=0.75)
    plt.ylim(mmranges[i-1])
    plt.xlim(mmranges[i-1])
    #plt.xticks(np.arange(1, 13), Time_months)
    ax.ticklabel_format(axis='both', style='sci', scilimits=(-3,1000))
    plt.title(plot_titles[i-1], fontsize=12.5, y=1)
    
    
plt.subplots_adjust(left=0.035, bottom=0.04, right=0.88, top=0.91, hspace=0.25, wspace=0.23) # the amount of height/width reserved for space between subplots

fig.legend(hs, models, loc='lower right', bbox_to_anchor=(0.999, 0.03)) 
plt.suptitle('CMIP6 models - Seasonality [(max-min)/ave] of North Pacific (x-axis) and North Atlantic (y-axis) - '+ str(start_year) + '-' + str(end_year), fontsize=18)

plt.savefig('NA_NP_max-min_ave.png', dpi=600)    


# pCO2 separation (model by model)


In [ ]:
n=0
for model in models:
    ofn = 'save/' + str(model) + '_pco2-t_np.txt'
    if os.path.exists(ofn):
        n = n +1

n

In [ ]:
%%time

## PAPA


fig = plt.figure(figsize=(18, 11))
ncol = 3
nrow = 4
i = 0
for model in models:
    
    f_pt = 'save/' + str(model) + '_pco2-t_np.txt'
    f_pnt = 'save/' + str(model) + '_pco2-nt_np.txt'
    f_p = 'save/' + str(model) + '_spco2_np.txt'
    
    if os.path.exists(f_pt):
        
        pco2_t = []
        with open(f_pt, "r") as rf:
            for line in rf:
                pco2_t.append(float(line.strip()))
        pco2_nt = []
        with open(f_pnt, "r") as rf:
            for line in rf:
                pco2_nt.append(float(line.strip()))
        pco2 = []
        with open(f_p, "r") as rf:
            for line in rf:
                pco2.append(float(line.strip()))
        
        xmon = np.arange(1, 13)
        
        i = i + 1
        ax = plt.subplot(ncol,nrow,i)
        
        plt.axhline(y=np.nanmean(pco2), color='k', linewidth=3.0)
        l1, = plt.plot(xmon, pco2, c='r', label='pCO2', 
                      marker='o', markersize=10, markerfacecolor='W', linewidth=4.0)
        l2, = plt.plot(xmon, pco2_t, c='b', label='pCO2-T', 
                      marker='o', markersize=10, markerfacecolor='W', linewidth=4.0)
        l3, = plt.plot(xmon, pco2_nt, c='g', label='pCO2-nonT',
                      marker='o', markersize=10, markerfacecolor='W', linewidth=4.0)
        plt.yticks(fontsize = 14)
        plt.ylim(270, 485)
        plt.xticks(xmon, Time_months, rotation=45, fontsize=10) # Setting the X-ticks ## rotation='horizontal' # rotation='vertical'    
        plt.title(str(model), fontsize=14)
    
plt.subplots_adjust(left=0.035, bottom=0.04, right=0.90, top=0.91, hspace=0.25, wspace=0.20) # the amount of height/width reserved for space between subplots

fig.legend([l1,l2,l3], ['pCO2', 'pCO2-T', 'pCO2-nonT'], loc='lower right', bbox_to_anchor=(0.8, 0.10)) 
plt.suptitle('CMIP6 SpCO2 separation (ppm) - North Pacific [45N-50N, 140W-150W] - '+ str(start_year) + '-' + str(end_year), fontsize=18)

plt.savefig('pCO2_sep_model_NP.png', dpi=600)    


In [ ]:
%%time

## NAPA

fig = plt.figure(figsize=(18, 11))
ncol = 3
nrow = 4
i = 0

for model in models:
    
    f_pt = 'save/' + str(model) + '_pco2-t_na.txt'
    f_pnt = 'save/' + str(model) + '_pco2-nt_na.txt'
    f_p = 'save/' + str(model) + '_spco2_na.txt'
    
    if os.path.exists(f_pt):
        
        pco2_t = []
        with open(f_pt, "r") as rf:
            for line in rf:
                pco2_t.append(float(line.strip()))
        pco2_nt = []
        with open(f_pnt, "r") as rf:
            for line in rf:
                pco2_nt.append(float(line.strip()))
        pco2 = []
        with open(f_p, "r") as rf:
            for line in rf:
                pco2.append(float(line.strip()))
        
        xmon = np.arange(1, 13)
        
        i = i + 1
        ax = plt.subplot(ncol,nrow,i)
        
        plt.axhline(y=np.nanmean(pco2), color='k', linewidth=3.0)
        l1, = plt.plot(xmon, pco2, c='r', label='pCO2', 
                      marker='o', markersize=10, markerfacecolor='W', linewidth=4.0)
        l2, = plt.plot(xmon, pco2_t, c='b', label='pCO2-T', 
                      marker='o', markersize=10, markerfacecolor='W', linewidth=4.0)
        l3, = plt.plot(xmon, pco2_nt, c='g', label='pCO2-nonT',
                      marker='o', markersize=10, markerfacecolor='W', linewidth=4.0)
        plt.yticks(fontsize = 14)
        plt.ylim(270, 485)
        plt.xticks(xmon, Time_months, rotation=45, fontsize=10) # Setting the X-ticks ## rotation='horizontal' # rotation='vertical'    
        plt.title(str(model), fontsize=14)
    
plt.subplots_adjust(left=0.035, bottom=0.04, right=0.90, top=0.91, hspace=0.25, wspace=0.20) # the amount of height/width reserved for space between subplots

fig.legend([l1,l2,l3], ['pCO2', 'pCO2-T', 'pCO2-nonT'], loc='lower right', bbox_to_anchor=(0.8, 0.10)) 
plt.suptitle('CMIP6 SpCO2 separation (ppm) - North Atlantic [45N-50N, 25W-35W] - '+ str(start_year) + '-' + str(end_year), fontsize=18)

plt.savefig('pCO2_sep_model_NA.png', dpi=600)    


# Zoo / biomass & Diatom / biomass

In [ ]:
for model in models:
    
    paray = get_data(df, 'phyc', model, expe, freq)
    if not isinstance(paray, xr.Dataset):
        continue
    
    n = 0
    zaray = get_data(df, 'zooc', model, expe, freq)
    daray = get_data(df, 'phydiat', model, expe, freq)
    if isinstance(zaray, xr.Dataset):
        n = n + 1
    if isinstance(daray, xr.Dataset):
        n = n + 1
    print(model, n)

In [ ]:
# temp test
models = ['CESM2', 'GFDL-CM4', 'GFDL-ESM4', 'CESM2-WACCM', 'IPSL-CM6A-LR', 'MRI-ESM2-0',
          'GISS-E2-1-G', 'GISS-E2-1-G-CC', 'UKESM1-0-LL',  'CanESM5', 'NorESM2-LM', 
          'MPI-ESM1-2-HR', 'NorCPM1', 'MIROC-ES2L']

In [ ]:
%%time

for model in models:
    
    paray = get_data(df, 'phyc', model, expe, freq)
    if not isinstance(paray, xr.Dataset):
        continue
    
    n = 0
    zaray = get_data(df, 'zooc', model, expe, freq)
    daray = get_data(df, 'phydiat', model, expe, freq)
    if isinstance(zaray, xr.Dataset):
        n = n + 1
    if isinstance(daray, xr.Dataset):
        n = n + 1
    
    if n == 0:
        continue            
            
    dp = paray.sel(time=slice(start_year, end_year))['phyc']
    
    if 'lev_partial' in dp.coords:
        dp = dp.isel(lev_partial=0)
    if 'olevel' in dp.coords:
        dp = dp.isel(olevel=0)
    if 'lev' in dp.coords:
        dp = dp.isel(lev=0)
    
    if 'latitude' in dp.coords:
        dp = dp.rename({'longitude': 'lon', 'latitude': 'lat'})
    if 'nav_lat' in dp.coords:
        dp = dp.rename({'nav_lon': 'lon', 'nav_lat': 'lat'})
    
    regridder = xe.Regridder(dp, ds_out, 'bilinear', periodic=True, ignore_degenerate=True)
    regridder._grid_in = None
    regridder._grid_out = None
    
    if isinstance(zaray, xr.Dataset):
        dz = zaray.sel(time=slice(start_year, end_year))['zooc']
        
        if 'lev_partial' in dz.coords:
            dz = dz.isel(lev_partial=0)
        if 'olevel' in dz.coords:
            dz = dz.isel(olevel=0)
        if 'lev' in dz.coords:
            dz = dz.isel(lev=0)
            
        if 'latitude' in dz.coords:
            dz = dz.rename({'longitude': 'lon', 'latitude': 'lat'})
        if 'nav_lat' in dz.coords:
            dz = dz.rename({'nav_lon': 'lon', 'nav_lat': 'lat'})
        
        zp_ratio = dz/dp
        
        
        zp_ratio_out = regridder(zp_ratio)
        zpr = xr.Dataset({'zpr':zp_ratio_out})
        
        if zpr['zpr'].shape[0]==240:
            #PAPA
            zpr_np = zpr.sel(lat=slice(45,50)).sel(lon=slice(210,220))
            
            mzpr_np = zpr_np['zpr'].mean(dim='time', skipna=1)
            anomzpr_np = zpr_np['zpr'] - mzpr_np
            anomzpr_np.load()
            anomzpr_detrended_np = xr.apply_ufunc(detrend, anomzpr_np, 
                                                 kwargs={'axis': 0}).where(~anomzpr_np.isnull())
            zpr_detrend_np = anomzpr_detrended_np + mzpr_np
            vzpr_np = zpr_detrend_np.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values
            
            savef_np = 'save/' + model + '_zpr_np.txt'
            with open(savef_np, 'w') as npf:
                for idata in vzpr_np:
                    npf.write(str(idata) +"\n")
            
            #NABA
            zpr_na = zpr.sel(lat=slice(45,50)).sel(lon=slice(325,335))
            mzpr_na = zpr_na['zpr'].mean(dim='time',skipna=1)
            anomzpr_na = zpr_na['zpr'] - mzpr_na
            anomzpr_na.load()
            anomzpr_detrended_na = xr.apply_ufunc(detrend, anomzpr_na,
                                                 kwargs={'axis': 0}).where(~anomzpr_na.isnull())
            zpr_detrend_na = anomzpr_detrended_na + mzpr_na
            vzpr_na = zpr_detrend_na.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values
            
            savef_na = 'save/' + model + '_zpr_na.txt'
            with open(savef_na, 'w') as naf:
                for idata in vzpr_na:
                    naf.write(str(idata) +"\n")
                        
    if isinstance(daray, xr.Dataset):
        dd = daray.sel(time=slice(start_year, end_year))['phydiat']
        
        if 'lev_partial' in dd.coords:
            dd = dd.isel(lev_partial=0)
        if 'olevel' in dd.coords:
            dd = dd.isel(olevel=0)
        if 'lev' in dz.coords:
            dd = dd.isel(lev=0)
            
        if 'latitude' in dd.coords:
            dd = dd.rename({'longitude': 'lon', 'latitude': 'lat'})
        if 'nav_lat' in dd.coords:
            dd = dd.rename({'nav_lon': 'lon', 'nav_lat': 'lat'})
        
        d_frac = dd/dp
        
        d_frac_out = regridder(d_frac)
        dfrac = xr.Dataset({'dfrac':d_frac_out})
        
        if dfrac['dfrac'].shape[0]==240:
            #PAPA
            df_np = dfrac.sel(lat=slice(45,50)).sel(lon=slice(210,220))
            mdf_np = df_np['dfrac'].mean(dim='time', skipna=1)
            anomdf_np = df_np['dfrac'] - mdf_np
            anomdf_np.load()
            anomdf_detrended_np = xr.apply_ufunc(detrend, anomdf_np, 
                                                 kwargs={'axis': 0}).where(~anomdf_np.isnull())
            df_detrend_np = anomdf_detrended_np + mdf_np
            vdf_np = df_detrend_np.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values
            
            savef_np = 'save/' + model + '_dfrac_np.txt'
            with open(savef_np, 'w') as npf:
                for idata in vdf_np:
                    npf.write(str(idata) +"\n")
            
            #NABA
            df_na = dfrac.sel(lat=slice(45,50)).sel(lon=slice(325,335))
            mdf_na = df_na['dfrac'].mean(dim='time',skipna=1)
            anomdf_na = df_na['dfrac'] - mdf_na
            anomdf_na.load()
            anomdf_detrended_na = xr.apply_ufunc(detrend, anomdf_na,
                                                 kwargs={'axis': 0}).where(~anomdf_na.isnull())
            df_detrend_na = anomdf_detrended_na + mdf_na
            vdf_na = df_detrend_na.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values
            
            savef_na = 'save/' + model + '_dfrac_na.txt'
            with open(savef_na, 'w') as naf:
                for idata in vdf_na:
                    naf.write(str(idata) +"\n")
    
    regridder.clean_weight_file()


In [ ]:
%%time

var2 = ['dfrac', 'zpr']
v2ranges = [[-0.01,4.1],[-0.01,4.1]]
v2titles = ['Diatom%', 'Zooplankton/Phytoplankton']

fig = plt.figure(figsize=(15, 12.5))
ncol = 4
nrow = 4
i = 0
for var in var2:
    i = i + 1
    
    ax = plt.subplot(ncol,nrow,i)
    
    mi = 0
    hs = []
    for model in models:
        
        ofnp = 'save/' + str(model) + '_' + str(var) + '_np.txt'
        ofna = 'save/' + str(model) + '_' + str(var) + '_na.txt'
        if os.path.exists(ofnp):
            if os.path.exists(ofna):
                
                np_data = []
                with open(ofnp, "r") as rfnp:
                    for line in rfnp:
                        np_data.append(float(line.strip()))
                        
                na_data = []
                with open(ofna, "r") as rfna:
                    for line in rfna:
                        na_data.append(float(line.strip()))
                        
                np_value = np.array(np_data)
                na_value = np.array(na_data)

                if np.shape(np_value)[0] > 0:
                    np_plot = (np.nanmax(np_value) - np.nanmin(np_value)) / np.nanmean(np_value)
                    na_plot = (np.nanmax(na_value) - np.nanmin(na_value)) / np.nanmean(na_value)
                    
                    h = plt.scatter(np_plot, na_plot, s=100, marker='o', edgecolor = 'black',
                            color=Colors[mi], label=str(model))
                else:
                    h = plt.scatter([], [], s=100, marker='o', edgecolor = 'black',
                            color=Colors[mi], label=str(model))
        else:
            h = plt.scatter([], [], s=100, marker='o', edgecolor = 'black',
                        color=Colors[mi], label=str(model))
            
        mi = mi +1
        hs.append(h)
    #plt.subplots_adjust(left=0.12, bottom=0.05, right=0.80, top=0.92, hspace=0.3, wspace=0.02)
    plt.plot([-1000,1000],[-1000,1000], 'k--', linewidth=0.75)
    plt.ylim(v2ranges[i-1])
    plt.xlim(v2ranges[i-1])
    #plt.xticks(np.arange(1, 13), Time_months)
    ax.ticklabel_format(axis='both', style='sci', scilimits=(-3,1000))
    plt.title(v2titles[i-1], fontsize=12.5, y=1)
    
    
plt.subplots_adjust(left=0.035, bottom=0.04, right=0.88, top=0.91, hspace=0.25, wspace=0.23) # the amount of height/width reserved for space between subplots

fig.legend(hs, models, loc='lower right', bbox_to_anchor=(0.999, 0.03)) 
plt.suptitle('CMIP6 models - Seasonality [(max-min)/ave] of North Pacific (x-axis) and North Atlantic (y-axis) - '+ str(start_year) + '-' + str(end_year), fontsize=18)

plt.savefig('NA_NP_max-min_ave_diaf_zoor.png', dpi=600)    


In [ ]:
%%time

var2 = ['dfrac', 'zpr']
v2ranges = [[-0.01,1.01],[-0.01,1.01]]
v2titles = ['Diatom %', 'Zooplankton / Phytoplankton']



fig = plt.figure(figsize=(15, 12.5))
ncol = 4
nrow = 4
i = 0
for var in var2:
    i = i + 1
    
    ax = plt.subplot(ncol,nrow,i)
    
    mi = 0
    hs = []
    for model in models:
        
        ofnp = 'save/' + str(model) + '_' + str(var) + '_np.txt'
        ofna = 'save/' + str(model) + '_' + str(var) + '_na.txt'
        if os.path.exists(ofnp):
            if os.path.exists(ofna):
                
                np_data = []
                with open(ofnp, "r") as rfnp:
                    for line in rfnp:
                        np_data.append(float(line.strip()))
                        
                na_data = []
                with open(ofna, "r") as rfna:
                    for line in rfna:
                        na_data.append(float(line.strip()))
                        
                np_value = np.array(np_data)
                na_value = np.array(na_data)
                
                h = plt.scatter(np.mean(np_value), np.mean(na_value), s=100, marker='o', edgecolor = 'black',
                            color=Colors[mi], label=str(model))
        else:
            h = plt.scatter([], [], s=100, marker='o', edgecolor = 'black',
                        color=Colors[mi], label=str(model))
            
        mi = mi +1
        hs.append(h)
    #plt.subplots_adjust(left=0.12, bottom=0.05, right=0.80, top=0.92, hspace=0.3, wspace=0.02)
    
    plt.ylim(v2ranges[i-1])
    plt.xlim(v2ranges[i-1])
    #plt.xticks(np.arange(1, 13), Time_months)
    ax.ticklabel_format(axis='both', style='sci', scilimits=(-2,1000))
    plt.title(v2titles[i-1], fontsize=12.5, y=1.05)
    plt.plot([-1000,1000],[-1000,1000], 'k--', linewidth=0.75)
    
    
plt.subplots_adjust(left=0.035, bottom=0.04, right=0.88, top=0.91, hspace=0.35, wspace=0.25) # the amount of height/width reserved for space between subplots

fig.legend(hs, models, loc='lower right', bbox_to_anchor=(0.999, 0.03)) 
plt.suptitle('CMIP6 models - Average value for North Pacific (x-axis) and North Atlantic (y-axis) - '+ str(start_year) + '-' + str(end_year), fontsize=18)

plt.savefig('NA_NP_average_diaf_zoor.png', dpi=600)    


In [ ]:
np.shape([])[0] > 0

In [ ]:
not isinstance(staray, xr.Dataset)

In [ ]:
x=np.asarray(range(len(a)))

In [ ]:
m_slope, b_intercept, r_val, p_val, std_err = stats.linregress(x, a)

In [ ]:
daa = m_slope*x + b_intercept
da=a-daa+maa

In [ ]:
da[:11]

In [ ]:
papa_detrended[:11,0,0]

In [ ]:
#plt.plot(x,da,'b')
plt.plot(x,papa_detrended[:,0,0],'black')
plt.plot(x,a,'r')
plt.plot(x, daa)

In [ ]:
PAPA_plot = []
for model in models:
    ds = papa_dic[model]
    try:
        ds = ds.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1)
        #print(ds)

    except Exception as e:
        print(model, e)
    PAPA_plot.append(ds)
PAPA_plot_dic = {key:value for key, value in zip(models, PAPA_plot)}

In [ ]:
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-90, 91, 1.0)),
                     'lon': (['lon'], np.arange(0, 361, 1.0))})

for var in var_names:
    xrays = []
    n = 0
    for model in models:
        xray = get_data(df, var, model, expe, freq)
        if isinstance(xray, xr.Dataset):
            n=n+1
        xrays.append(xray)
    if n > 0:
        xray_dic = {key:value for key, value in zip(models, xrays)}
        
        regrids = []
        for model in models:
            if isinstance(xray_dic[model], xr.Dataset):
                ds = xray_dic[model].sel(time=slice(start_year, end_year))[var]  
                if 'latitude' in ds.coords:
                    ds = ds.rename({'longitude': 'lon', 'latitude': 'lat'})
                if 'nav_lat' in ds.coords:
                    ds = ds.rename({'nav_lon': 'lon', 'nav_lat': 'lat'})
                
                dsr = xe.Regridder(ds, ds_out, 'bilinear', periodic=True, ignore_degenerate=True)
                dsr._grid_in = None
                dsr._grid_out = None
                dsr_out0 = dsr(ds)
                dsr_out = xr.Dataset({var:dsr_out0})
                dsr.clean_weight_file()        
            else:
                dsr_out = []      
            regrids.append(dsr_out)   
        regrid_dic = {key:value for key, value in zip(models, regrids)}
        
        papa_aray = []
        for model in models:
            ds = regrid_dic[model]
            
            if isinstance(ds, xr.Dataset):
                if 'lev_partial' in ds.coords:
                    ds = ds.isel(lev_partial=0)
                if 'olevel' in ds.coords:
                    ds = ds.isel(olevel=0)
                if 'lev' in ds.coords:
                    ds = ds.isel(lev=0)     
                
                if ds[var].shape[0]==240:
                    ds_r = ds.sel(lat=slice(45,50)).sel(lon=slice(220,230))
                    dsm = ds_r[var].mean(dim='time')
                    ds_anom = ds_r[var] - dsm
                    ds_anom.load()
                    ds_anom_detrended = xr.apply_ufunc(detrend, ds_anom, kwargs={'axis': 0}).where(~ds_anom.isnull())
                    ds_detrend = ds_anom_detrended + dsm
                    ds = ds_detrend.groupby('time.month').mean('time',skipna=1).mean('lon',skipna=1).mean('lat',skipna=1).values
                else:
                    ds = []
                    #print(model, regrid_dic[model])       
            papa_aray.append(ds)
        papa_dic = {key:value for key, value in zip(models, papa_aray)}
        
        save_var_file = var + '_papa.txt'
        with open(save_var_file, 'w') as sf:
            for aray in papa_aray:
                sf.write(str(aray))
                
        plt.figure(figsize=(8.5, 5))
        l = []
        for model in models:  
            if isinstance(papa_dic[model], list):
                continue
            else:
                s = papa_dic[model]
                if model == 'CNRM-ESM2-1' or model == 'IPSL-CM6A-LR':
                    if var == 'chl':
                        s = papa_dic[model] * 0.001        # need more infromation
                if model == 'GISS-E2-1-G' or model == 'GISS-E2-1-G-CC':
                    if var == 'intpp':
                        s = -papa_dic[model]               # need more infromation
                    if var == 'fgco2':
                        s = papa_dic[model]*(10**(-10))    # need more infromation       
                plt.plot(np.arange(1, 13), s, label=str(model), marker='o', markersize=7, markerfacecolor='W', linewidth=2.0)
                l.append(str(model))
        plt.subplots_adjust(left=0.12, bottom=0.05, right=0.80, top=0.92, hspace=0.3, wspace=0.02)
        plt.xticks(np.arange(1, 13), Time_months)
        plt.legend(l,loc='center left', bbox_to_anchor=(1, 0.5))
        savename = var + '_papa.png'
        plt.savefig(savename, dpi=600)   
        plt.close() 

In [ ]:
client.close()
cluster.close()